In [12]:
import lxml.html
import lxml.etree
import requests
#from tqdm import tqdm
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Question(Base):
    __tablename__ = 'questions'
    id = Column(Integer, primary_key=True)
    question = Column(Text, nullable=False)
    answer = Column(Text, nullable=False)
    
    def __init__(self, text, answer):
        self.question = text
        self.answer = answer

engine = create_engine('sqlite:///chgk.db')
Base.metadata.bind = engine
Base.metadata.create_all()

db_session = sessionmaker(bind=engine)()

def moveNextPage(link):
    return lxml.html.fromstring(requests.get('https://db.chgk.info/' + link).text)

#response = requests.get('https://db.chgk.info')
#tree = lxml.html.fromstring(response.text)

tree = moveNextPage('')
nextPageIsExist = True
pbar = tqdm(total=int(9e9))
while nextPageIsExist:
    urles = tree.xpath('//table[@class="last_packages"]//td[2]/a/@href')
    for url in urles:
        for q in moveNextPage(url).xpath('//div[@class="question"]'):
            question = ''.join(q.xpath('p/text()')).replace('\n', '')
            answer = ''.join(q.xpath('.//p/strong[@class="Answer"]/../text()')).replace('/n', '')
            db_session.add(Question(question, answer))
            pbar.update(1)
    db_session.commit()
            
    nextPage = tree.xpath('//li[@class="pager-next"]')
    if nextPage:
        tree = moveNextPage(nextPage[0].xpath('a/@href')[0])
    else:
        nextPageIsExist = False
pbar.close()
print('Done')


  0%|                                                              | 278067/9000000000 [2:39:15<85910:26:49, 29.10it/s]
